In [1]:
import os
import pandas as pd
import numpy as np
import json

In [9]:
df = pd.read_json('rules_valid.json')
df.T

,p,labels,given
1,P462,[RoboCop],sbj
2,P57,[RoboCop],sbj
3,P17,[Event Cinemas],sbj
4,P58,[Jim Sharman],obj
5,P57,[Shock Treatment and The Rocky Horror Picture ...,sbj
...,...,...,...
49663,P688,[tetracenomycin polyketide synthesis O-methylt...,obj
49665,P681,[Synapse differentiation-inducing gene protein 1],sbj
49666,P1196,[H. David Dalquist],sbj
49667,P735,[H. David Dalquist],sbj


In [30]:
rule_ids = list(df.head(0))
rule_preds = list(df.T["p"])

In [7]:
path = "/mount/arbeitsdaten31/studenten1/dialog-system/2020/student_directories/Chang_Wentseng/"
df_dev = pd.read_json(path + 'data/csqa.dev.json')
df_dev

,id,tokens,relation,topic_tags,direction
0,1,"[[CLS], which, color, is, robocop, associated,...",P462,"[O, O, O, O, B_SBJ, O, O, O, O]",out
1,2,"[[CLS], which, person, directed, robocop, ?, [...",P57,"[O, O, O, O, B_SBJ, O, O]",out
2,3,"[[CLS], which, political, territory, is, event...",P17,"[O, O, O, O, O, B_SBJ, I_SBJ, O, O, O, O, O]",out
3,4,"[[CLS], which, works, of, art, are, jim, sharm...",P58,"[O, O, O, O, O, O, B_OBJ, I_OBJ, O, O, O, O, O]",in
4,5,"[[CLS], who, directed, shock, treatment, and, ...",P57,"[O, O, O, B_SBJ, I_SBJ, O, B_SBJ, I_SBJ, I_SBJ...",out
...,...,...,...,...,...
45071,49663,"[[CLS], what, is, the, nucleic, acid, sequence...",P688,"[O, O, O, O, O, O, O, O, O, O, O, B_OBJ, I_OBJ...",in
45072,49665,"[[CLS], which, cellular, components, are, comp...",P681,"[O, O, O, O, O, O, O, B_SBJ, I_SBJ, I_SBJ, I_S...",out
45073,49666,"[[CLS], what, situation, did, h, ., david, dal...",P1196,"[O, O, O, O, B_SBJ, I_SBJ, I_SBJ, I_SBJ, O, O,...",out
45074,49667,"[[CLS], what, is, forename, of, h, ., david, d...",P735,"[O, O, O, O, O, B_SBJ, I_SBJ, I_SBJ, I_SBJ, O, O]",out


In [33]:
gold_ids = list(df_dev["id"])
gold_preds = list(df_dev["relation"])

In [43]:
clean_ids = []
clean_preds = []
for ids in rule_ids:
    if ids in gold_ids:
        clean_ids.append(ids)
        clean_preds.append(df[ids][0])
print(len(clean_ids))
print(len(clean_preds))
        

45076
45076


In [44]:
d = {"rule_ids":clean_ids, "rule_preds":clean_preds, "gold_ids":gold_ids, "gold_preds":gold_preds}
compare = pd.DataFrame(data=d)
compare

,rule_ids,rule_preds,gold_ids,gold_preds
0,1,P462,1,P462
1,2,P57,2,P57
2,3,P17,3,P17
3,4,P58,4,P58
4,5,P57,5,P57
...,...,...,...,...
45071,49663,P688,49663,P688
45072,49665,P681,49665,P681
45073,49666,P1196,49666,P1196
45074,49667,P735,49667,P735


In [115]:
drop_compare = compare.dropna()
drop_compare

,rule_ids,rule_preds,gold_ids,gold_preds
0,1,P462,1,P462
1,2,P57,2,P57
2,3,P17,3,P17
3,4,P58,4,P58
4,5,P57,5,P57
...,...,...,...,...
45071,49663,P688,49663,P688
45072,49665,P681,49665,P681
45073,49666,P1196,49666,P1196
45074,49667,P735,49667,P735


In [46]:
tagset = pd.read_json(path + 'data/csqa_tags.json')
tagset

,0
0,P6
1,P7
2,P9
3,P16
4,P17
...,...
276,P3179
277,P3195
278,P3279
279,P3301


In [120]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
new_tagset = list(tagset[0])
le.fit(new_tagset)
gp = le.transform(drop_compare["gold_preds"])
gp
#list(le.inverse_transform(gp))

array([183, 207,  76, ...,  26, 241,  24])

tr_none = []
for i in d["rule_preds"]:
    if i == None:
        tr_none.append(str(i))
    else:
        tr_none.append(i)
tr_none

In [121]:
rp = le.transform(drop_compare["rule_preds"])
print(len(rp))
rp
#list(le.inverse_transform(rp))

44438


array([183, 207,  76, ...,  26, 241,  24])

In [122]:
from sklearn.metrics import f1_score
gold_preds = gp
rule_preds = rp
f1_score(gold_preds, rule_preds, average='macro')

0.970758933696582

In [130]:
uniq_tagset = []
for i in drop_compare["rule_preds"]:
    if i not in uniq_tagset:
        uniq_tagset.append(i)
len(uniq_tagset)

256

In [132]:
from sklearn.metrics import classification_report
gold_preds = gp
rule_preds = rp
uniq_tagset = uniq_tagset
print(classification_report(gold_preds, rule_preds, target_names=uniq_tagset))

              precision    recall  f1-score   support

        P462       1.00      1.00      1.00       246
         P57       1.00      1.00      1.00       245
         P17       1.00      0.99      1.00       775
         P58       1.00      1.00      1.00        35
        P682       1.00      0.57      0.73         7
        P703       1.00      1.00      1.00        46
        P161       1.00      1.00      1.00         2
        P176       1.00      1.00      1.00        60
        P264       0.95      1.00      0.97        69
        P750       1.00      1.00      1.00        36
        P136       0.93      0.96      0.95        27
        P123       1.00      1.00      1.00      2825
         P27       0.89      1.00      0.94        90
        P991       1.00      0.96      0.98        95
         P21       1.00      1.00      1.00       367
        P106       1.00      0.57      0.73         7
        P921       1.00      0.78      0.88         9
       P1412       1.00    

In [133]:
f = open('rulebase_report','w')
print(classification_report(gold_preds, rule_preds, target_names=uniq_tagset), file=f)